1. Z bazy danych znajdującej się w pliku data/sql/titanic.db, pobierz dane wszystkich osób, które przeżyły katastrofę Titanica i w czasie katastrofy miały powyżej 35 lat.
2. Na stronie https://en.wikipedia.org/wiki/Passengers_of_the_Titanic znajdź wszystkie osoby, które przeżyły. Pobierz linki do stron Wikipedii o tych osobach.
3. Połącz dane z naszej bazy danych z danymi pobranymi z Wikipedii. Interesują nas tylko osoby znajdujące się w obu zbiorach danych (ich nazwiska zostały zapisane w identyczny sposób w obu listach).

## Dla każdej osoby:
1. Otwórz stronę Wikipedii o tej osobie.
2. Z prawej strony znajduje się prostokąt z podsumowaniem informacji o osobie. Pobierz z niego wszystkie dane.

## Po pobraniu wszystkich osób:
1. Zapisz dane z wikipedii i bazy danych w jednym pliku XLS. Dane o każdej osobie powinny znaleźć się w osobnej zakładce (nazwanej nazwiskiem i imieniem tej osoby).
2. Każda zakładka powinna zawierać dwie kolumny: nazwa atrybutu i wartość (bez nagłówków i bez indeksu)


In [99]:
import pandas as pd
import sqlite3
import requests
from bs4 import BeautifulSoup

In [100]:
file = "./titanic.db"
cnx = sqlite3.connect(file)
query = """SELECT * FROM passengers as p LEFT JOIN survivors as s ON p.PassengerId=s.PassengerId
LEFT JOIN tickets as t ON p.PassengerId=t.PassengerId
WHERE Survived=1 AND Age >35"""

try:
    titanic_df = pd.read_sql_query(query, cnx)
    
finally:
    cnx.close()

In [159]:
survivor_names = titanic_df['Name'].tolist()

In [107]:
URL = 'https://en.wikipedia.org/wiki/Passengers_of_the_Titanic'

response = requests.get(URL)
soup = BeautifulSoup(response.text, 'html.parser')

html = soup.select('tr[style="background:#9bddff;"] > td:first-child > a')

people_urls = []

for i in html:
    links = i.text
    if links in survivor_names:
        people_urls.append((f"https://en.wikipedia.org/{i['href']}", links))

In [108]:
for link, name in people_urls:
    print(link, name)

In [160]:
people_informations = []

for link_name, name_people in people_urls:

    response = requests.get(link_name)
    soup = BeautifulSoup(response.text, 'html.parser')

    df = pd.read_html(str(soup.find_all('table', {'class': 'infobox'})))[0]
    df.columns = ['Attribute', 'Value']
    
    data_from_db = titanic_df[titanic_df['Name'] == name_people].transpose().reset_index()
    data_from_db.columns = ['Attribute', 'Value']
    
    
    df = pd.concat([df, data_from_db])
    df[df['Attribute'] != 'index']
    people_informations.append((df, name_people))

C:\Users\milen\AppData\Local\Temp\ipykernel_2288\3592993630.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup.find_all('table', {'class': 'infobox'})))[0]
C:\Users\milen\AppData\Local\Temp\ipykernel_2288\3592993630.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup.find_all('table', {'class': 'infobox'})))[0]
C:\Users\milen\AppData\Local\Temp\ipykernel_2288\3592993630.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup.find_all('table', {'class': 'infobox'})))[0]


In [158]:
writer = pd.ExcelWriter('titanic.xlsx', engine='xlsxwriter' )

for df, name in people_informations:
    df.to_excel(writer, sheet_name=name, index=False)
